In [1]:
import os
os.chdir('/content/drive/MyDrive/yeardream_competition/main2_NLP_myself')
!pwd

/content/drive/MyDrive/yeardream_competition/main2_NLP_myself


In [2]:
import pandas as pd

import re
import ast

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

import pickle
import json

In [3]:

train = pd.read_csv('train_data.csv')

train['cluster'] = 0




train['0'] = 0
train['1'] = 0
train['2'] = 0
train['3'] = 0


train['choices'] = 0
# 선택지 개수를 알려주는 컬럼



In [4]:
# 문제 clustering
## '문제' 컬럼이 '?'로 끝나는 행들의 'cluster' 값을 1로 변경
train.loc[train['문제'].str.endswith('?'), 'cluster'] = 1
## '문제' 컬럼이 '시나리오 1 |'을 포함하는 행들의 'cluster' 값을 2로 변경
train.loc[train['문제'].str.contains('시나리오 1 |', regex=False), 'cluster'] = 2
## '문제' 컬럼이 '___'을 포함하는 행들의 'cluster' 값을 3으로 변경 -> 빈공간을 채워야 함
train.loc[train['문제'].str.contains('___', regex=False), 'cluster'] = 3
## 명제(진술)이 참인지 거짓인지 판단하는 문제
train.loc[train['문제'].str.contains('명제 1 |',regex=False), 'cluster'] = 4

train['cluster'].value_counts()

,count
cluster,
1,4821
3,1784
0,1727
2,627
4,41


In [5]:
train.head()

,문제,선택지,답안,cluster,0,1,2,3,choices
0,다음 중 상관 계수 r의 값을 변경할 수 있는 것은 무엇입니까?,"['측정 단위의 변경', '어느 변수를 x로, 어느 변수를 y로 부르는지의 변경',...",3,1,0,0,0,0,0
1,한 건물 소유자가 도시에 있는 큰 건물을 소유하고 있었습니다. 소유자는 건물을 상인...,"['건물 소유자와 상인 간에는 부동산 소유권과 계약상의 관계가 존재합니다.', '상...",2,1,0,0,0,0,0
2,유전율이 4.0인 비자성 유전체 내부에서의 빛의 속도는 _______입니다.,"['1.2 x 10^9 m/s', '3.0 x 10^8 m/s', '1.5 x 10...",2,3,0,0,0,0,0
3,다음 중 완전 경쟁 하에서 가격(P)이 한계 수익(MR)과 같은 이유와 독점 또는 ...,"['I, II, III.', 'I와 II.', 'I만.', 'I와 III.']",1,0,0,0,0,0,0
4,다음 중 어떤 것이 일부 지역에서 일반적으로 큰 조수 범위(최대 60피트)가 있고 ...,"['태양의 위치가 지역마다 다릅니다.', '코리올리 효과와 지구의 자전이 고위도에서...",2,1,0,0,0,0,0


In [6]:
train['cluster'].value_counts()

,count
cluster,
1,4821
3,1784
0,1727
2,627
4,41


In [7]:
train['선택지'].tolist()

["['측정 단위의 변경', '어느 변수를 x로, 어느 변수를 y로 부르는지의 변경', 'x 변수의 모든 값에 동일한 상수를 더하는 것', '위의 어느 것도 r 값을 변경할 수 없다.']",
 "['건물 소유자와 상인 간에는 부동산 소유권과 계약상의 관계가 존재합니다.', '상인과 법인 간에는 부동산 소유권과 계약상의 관계가 존재합니다.', '건물 소유자와 법인 간에는 부동산 소유권 관계는 존재하지만 계약상의 관계는 존재하지 않습니다.', '건물 소유자와 비영리 단체 간에는 부동산 소유권 관계도 계약상의 관계도 존재하지 않습니다.']",
 "['1.2 x 10^9 m/s', '3.0 x 10^8 m/s', '1.5 x 10^8 m/s', '1.0 x 10^8 m/s']",
 "['I, II, III.', 'I와 II.', 'I만.', 'I와 III.']",
 "['태양의 위치가 지역마다 다릅니다.', '코리올리 효과와 지구의 자전이 고위도에서 조수 흐름을 증대시키는 경향이 있습니다.', '해저 지형과 해안선의 모양이 특정 지역에서 조수 흐름을 증폭시키는 역할을 합니다.', '무역풍이 물을 바위 해안 근처의 큰 조수 융기로 밀어냅니다.']",
 "['시드라만', '오만만', '홍해', '페르시아만']",
 "['촉각', '시각', '미각', '청각']",
 "['후속 세대의 개체군은 성공에 더 적합한 특성을 가진 개체의 비율이 더 높아야 합니다.', '개별 유기체는 일생 동안 진화를 겪습니다.', '서식지는 일반적으로 무한한 자원을 가지고 있지 않습니다.', '자연 개체군은 일반적으로 그들의 생물학적 잠재력에 따라 번식하는 경향이 있습니다.']",
 "['잘못, 잘못', '잘못, 잘못 아님', '잘못 아님, 잘못', '잘못 아님, 잘못 아님']",
 "['발열.', '비가습 산소 사용.', '과도한 체중 감소.', '설사.']",
 "['탈분극', '재분극', '과분극', '활동 전위']",
 "['계절성 정동 장애', '폐소공포증', '알츠하이머병', '긴장성

In [10]:

p_data = []

for index, row in train.iterrows():
    q = row['문제']

    try:
        # 전처리된 선택지 문자열을 안전하게 파싱
        opt = ast.literal_eval(row['선택지'])
    except Exception as e:
        print(f'{index} error 발생 : {e}')
        print("원본 선택지:", row['선택지'])
        # 에러 발생 시 원본 선택지를 그대로 사용
        opt = row['선택지']

    cls = row['cluster']
    ans = row['답안']
    p_data.append({
        "문제": q,
        "cluster": cls,
        "선택지": opt,
        '답안': ans
    })

# 결과 데이터프레임 생성
df_p_data = pd.DataFrame(p_data)
df_p_data.info()

2943 error 발생 : invalid syntax. Perhaps you forgot a comma? (<unknown>, line 1)
원본 선택지: ['국가가 안정성을 제공하지 못함으로써 사회의 각 구성 요소가 자신의 안녕을 유지하기 위해 경쟁하게 되어 불안을 초래하는 상황을 말합니다. 이 조건은 자가 증식적이며, 체제를 확보하기 위한 조치가 더 큰 저항을 불러일으키기 때문에 반영구적인 긴급 무정부 상태가 됩니다.', '약소국의 불안정 딜레마는 주로 외부 조건으로 인해 발생하며, 구조적 무정부 상태와 유사한 상황을 초래합니다. 약소국이 자국의 지역적 입지를 개선하기 위한 조치를 취할 때 지역 내 불안을 조성합니다.', '약소국의 불안정 딜레마는 사회의 각 구성 요소가 자신의 안녕과 이익을 보호하고 유지하기 위한 경쟁에서 비롯됩니다. 그러나 지배 엘리트는 사회적 경쟁 영역과 분리되어 정책 딜레마를 초래합니다. 질서를 회복하기 위해 폭력 수단을 사용할 경우 체제의 기반이 약화됩니다.', '약소국의 불안정 딜레마는 정치적 및 제도적 중심화와 힘의 독점이 부족한 상황에서 발생합니다. 그러나 제도를 강화하기 위해 무력을 동원하면 이 과정을 중단시킬 수 있습니다. '국가성'을 육성하지 못하는 것은 폭력 사용으로 반전됩니다. 사회적 불안은 긴급하지만 개발되지 않은 무정부 상태의 반영구적 상황입니다.']
3257 error 발생 : unterminated string literal (detected at line 1) (<unknown>, line 1)
원본 선택지: ['모든 RNA가 공통된 3' 말단에서 끝나고 중첩된 세트 전사체를 생성합니다', '긴 RNA 유전체와의 재조합을 이용합니다', '변이율이 높지 않습니다', '캡이 씌워진 세포 mRNA를 사용합니다']
4411 error 발생 : invalid syntax. Perhaps you forgot a comma? (<unknown>, line 1)
원본 선택지: ['정치적 편견 없이 진실되고 정확한 설명이어

In [41]:
df_p_data['0'] = df_p_data['선택지'].apply(lambda x: x[0].split(",")[0] if isinstance(x, list) and len(x) > 0 else None)
df_p_data['1'] = df_p_data['선택지'].apply(lambda x: x[1].split(",")[0] if isinstance(x, list) and len(x) > 1 else None)
df_p_data['2'] = df_p_data['선택지'].apply(lambda x: x[2].split(",")[0] if isinstance(x, list) and len(x) > 2 else None)
df_p_data['3'] = df_p_data['선택지'].apply(lambda x: x[3].split(",")[0] if isinstance(x, list) and len(x) > 3 else None)

In [42]:
df_p_data[['선택지', '0', '1', '2','3']]

,선택지,0,1,2,3
0,"[측정 단위의 변경, 어느 변수를 x로, 어느 변수를 y로 부르는지의 변경, x 변...",측정 단위의 변경,어느 변수를 x로,x 변수의 모든 값에 동일한 상수를 더하는 것,위의 어느 것도 r 값을 변경할 수 없다.
1,"[건물 소유자와 상인 간에는 부동산 소유권과 계약상의 관계가 존재합니다., 상인과 ...",건물 소유자와 상인 간에는 부동산 소유권과 계약상의 관계가 존재합니다.,상인과 법인 간에는 부동산 소유권과 계약상의 관계가 존재합니다.,건물 소유자와 법인 간에는 부동산 소유권 관계는 존재하지만 계약상의 관계는 존재하지...,건물 소유자와 비영리 단체 간에는 부동산 소유권 관계도 계약상의 관계도 존재하지 않...
2,"[1.2 x 10^9 m/s, 3.0 x 10^8 m/s, 1.5 x 10^8 m/...",1.2 x 10^9 m/s,3.0 x 10^8 m/s,1.5 x 10^8 m/s,1.0 x 10^8 m/s
3,"[I, II, III., I와 II., I만., I와 III.]",I,I와 II.,I만.,I와 III.
4,"[태양의 위치가 지역마다 다릅니다., 코리올리 효과와 지구의 자전이 고위도에서 조수...",태양의 위치가 지역마다 다릅니다.,코리올리 효과와 지구의 자전이 고위도에서 조수 흐름을 증대시키는 경향이 있습니다.,해저 지형과 해안선의 모양이 특정 지역에서 조수 흐름을 증폭시키는 역할을 합니다.,무역풍이 물을 바위 해안 근처의 큰 조수 융기로 밀어냅니다.
...,...,...,...,...,...
8995,"[제로, 사실상 제로, 지구 표면보다 상당히 약하다, 원형 경로를 따라 힘의 성분이...",제로,사실상 제로,지구 표면보다 상당히 약하다,원형 경로를 따라 힘의 성분이 없기 때문이다
8996,"[가시광선 펄스, 라디오파 신호, 일련의 적외선 펄스, 일련의 강한 적외선 빔]",가시광선 펄스,라디오파 신호,일련의 적외선 펄스,일련의 강한 적외선 빔
8997,"[10, 45, 125, 25]",10,45,125,25
8998,"[한편으로 자연주의는 보편적이고 객관적인 범위의 규칙들에 기초하고, 다른 한편으로 ...",한편으로 자연주의는 보편적이고 객관적인 범위의 규칙들에 기초하고,자연주의는 인권법 이론에만 봉사하는 반면,자연주의는 국제 사회의 근본적인 가치를 다루고,자연주의는 유럽 외부에서 옹호되는 사상 학파이며


In [43]:
df_p_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   문제       9000 non-null   object
 1   cluster  9000 non-null   int64 
 2   선택지      9000 non-null   object
 3   답안       9000 non-null   int64 
 4   0        8994 non-null   object
 5   1        8994 non-null   object
 6   2        8994 non-null   object
 7   3        8993 non-null   object
dtypes: int64(2), object(6)
memory usage: 562.6+ KB


In [57]:
def process_choice(choice):
    # 만약 choice가 리스트라면 join을 사용해 문자열로 변환
    if isinstance(choice, list):
        choice = ','.join(choice)

    # 1차로 쉼표 기준으로 나눕니다
    parts = choice.split(',')

    # 만약 4개의 부분으로 나눠졌다면, 그대로 반환
    if len(parts) == 4:
        return parts

    # 만약 4개보다 적다면, 부족한 부분을 '0'으로 채웁니다
    elif len(parts) < 4:
        return parts + ['0'] * (4 - len(parts))

    # 만약 4개보다 많다면, 정규표현식을 적용하여 다시 나눕니다
    else:
        parts = re.split(r"[(?<!['\"]) ?,(?!['\"])]", choice)
        # 결과가 4개 미만이라면 남은 부분을 0으로 채웁니다.
        if len(parts) < 4:
            return parts + ['0'] * (4 - len(parts))
        return parts[:4]

# '선택지' 컬럼에 대해 함수를 적용하고, 결과를 4개의 컬럼으로 나눕니다
df_p_data[['0', '1', '2', '3']] = df_p_data['선택지'].apply(process_choice).apply(pd.Series)


In [61]:
# 함수 정의: 작은 따옴표 또는 큰 따옴표 사이에 있는 쉼표를 하이픈으로 대체
def replace_commas_inside_quotes(text):
    if isinstance(text, str):  # 입력이 문자열인지 확인
        return re.sub(r"(['\"]).*?\1", lambda m: m.group(0).replace(',', '-'), text)
    else:
        return text  # 문자열이 아닌 경우에는 원래 값을 반환

# '선택지' 컬럼에 함수를 적용하여 변환
df_p_data['__선택지'] = df_p_data['선택지'].apply(replace_commas_inside_quotes)

In [73]:
df_p_data['__선택지'][22][3]

'기념비적 건축물이나 도로의 증거가 없었다.'

In [75]:
for i in range(df_p_data.shape[0]):
  try:
    df_p_data['0'][i] = df_p_data['__선택지'][i][0]
  except:
    df_p_data['0'][i] = '0'

  try:
    df_p_data['1'][i] = df_p_data['__선택지'][i][1]
  except:
    df_p_data['1'][i] = '0'

  try:
    df_p_data['2'][i] = df_p_data['__선택지'][i][2]
  except:
    df_p_data['2'][i] = '0'

  try:
    df_p_data['3'][i] = df_p_data['__선택지'][i][3]
  except:
    df_p_data['3'][i] = '0'

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
<ipython-input-75-c73691f2c281>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_p_data['0'][i] = df_p_data['__선택지'][i][0]
<ipython-input-75-c73691f2c281>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_p_data['1'][i] = df_p_data['__선택지'][i][1]
<ipython-input-75-c73691f2c281>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_p_data['2'][i] = df_p_data['__선택지'][i][2]
<ipython-input-75-c73691

In [76]:
df_p_data

,문제,cluster,선택지,답안,0,1,2,3,__선택지
0,다음 중 상관 계수 r의 값을 변경할 수 있는 것은 무엇입니까?,1,"[측정 단위의 변경, 어느 변수를 x로, 어느 변수를 y로 부르는지의 변경, x 변...",3,측정 단위의 변경,"어느 변수를 x로, 어느 변수를 y로 부르는지의 변경",x 변수의 모든 값에 동일한 상수를 더하는 것,위의 어느 것도 r 값을 변경할 수 없다.,"[측정 단위의 변경, 어느 변수를 x로, 어느 변수를 y로 부르는지의 변경, x 변..."
1,한 건물 소유자가 도시에 있는 큰 건물을 소유하고 있었습니다. 소유자는 건물을 상인...,1,"[건물 소유자와 상인 간에는 부동산 소유권과 계약상의 관계가 존재합니다., 상인과 ...",2,건물 소유자와 상인 간에는 부동산 소유권과 계약상의 관계가 존재합니다.,상인과 법인 간에는 부동산 소유권과 계약상의 관계가 존재합니다.,건물 소유자와 법인 간에는 부동산 소유권 관계는 존재하지만 계약상의 관계는 존재하지...,건물 소유자와 비영리 단체 간에는 부동산 소유권 관계도 계약상의 관계도 존재하지 않...,"[건물 소유자와 상인 간에는 부동산 소유권과 계약상의 관계가 존재합니다., 상인과 ..."
2,유전율이 4.0인 비자성 유전체 내부에서의 빛의 속도는 _______입니다.,3,"[1.2 x 10^9 m/s, 3.0 x 10^8 m/s, 1.5 x 10^8 m/...",2,1.2 x 10^9 m/s,3.0 x 10^8 m/s,1.5 x 10^8 m/s,1.0 x 10^8 m/s,"[1.2 x 10^9 m/s, 3.0 x 10^8 m/s, 1.5 x 10^8 m/..."
3,다음 중 완전 경쟁 하에서 가격(P)이 한계 수익(MR)과 같은 이유와 독점 또는 ...,0,"[I, II, III., I와 II., I만., I와 III.]",1,"I, II, III.",I와 II.,I만.,I와 III.,"[I, II, III., I와 II., I만., I와 III.]"
4,다음 중 어떤 것이 일부 지역에서 일반적으로 큰 조수 범위(최대 60피트)가 있고 ...,1,"[태양의 위치가 지역마다 다릅니다., 코리올리 효과와 지구의 자전이 고위도에서 조수...",2,태양의 위치가 지역마다 다릅니다.,코리올리 효과와 지구의 자전이 고위도에서 조수 흐름을 증대시키는 경향이 있습니다.,해저 지형과 해안선의 모양이 특정 지역에서 조수 흐름을 증폭시키는 역할을 합니다.,무역풍이 물을 바위 해안 근처의 큰 조수 융기로 밀어냅니다.,"[태양의 위치가 지역마다 다릅니다., 코리올리 효과와 지구의 자전이 고위도에서 조수..."
...,...,...,...,...,...,...,...,...,...
8995,원형 지구 궤도의 위성은 일정한 속도를 유지합니다. 그 이유는 중력이 _______...,3,"[제로, 사실상 제로, 지구 표면보다 상당히 약하다, 원형 경로를 따라 힘의 성분이...",3,제로,사실상 제로,지구 표면보다 상당히 약하다,원형 경로를 따라 힘의 성분이 없기 때문이다,"[제로, 사실상 제로, 지구 표면보다 상당히 약하다, 원형 경로를 따라 힘의 성분이..."
8996,한 사람이 리모컨을 사용하여 텔레비전의 볼륨 수준을 변경합니다. 리모컨에서 텔레비전...,3,"[가시광선 펄스, 라디오파 신호, 일련의 적외선 펄스, 일련의 강한 적외선 빔]",2,가시광선 펄스,라디오파 신호,일련의 적외선 펄스,일련의 강한 적외선 빔,"[가시광선 펄스, 라디오파 신호, 일련의 적외선 펄스, 일련의 강한 적외선 빔]"
8997,직각삼각형의 빗변이 10인치이고 한 각이 $45^{\circ}$입니다. 삼각형의 면...,1,"[10, 45, 125, 25]",3,10,45,125,25,"[10, 45, 125, 25]"
8998,자연주의와 실증주의의 주요 차이점은 무엇입니까?,1,"[한편으로 자연주의는 보편적이고 객관적인 범위의 규칙들에 기초하고, 다른 한편으로 ...",0,"한편으로 자연주의는 보편적이고 객관적인 범위의 규칙들에 기초하고, 다른 한편으로 실...","자연주의는 인권법 이론에만 봉사하는 반면, 실증주의는 더 넓고 일반적인 범위를 가진다","자연주의는 국제 사회의 근본적인 가치를 다루고, 실증주의는 국제 규칙 그 자체를 다룬다","자연주의는 유럽 외부에서 옹호되는 사상 학파이며, 실증주의는 유럽 중심의 이론이다","[한편으로 자연주의는 보편적이고 객관적인 범위의 규칙들에 기초하고, 다른 한편으로 ..."


In [78]:
df_p_data['문항개수'] = df_p_data['__선택지'].apply(lambda x: len(x))

In [83]:
df_p_data[df_p_data['문항개수'] != 4]

,문제,cluster,선택지,답안,0,1,2,3,__선택지,문항개수
2943,약소국이 직면한 '안보 딜레마'는 무엇입니까?,1,['국가가 안정성을 제공하지 못함으로써 사회의 각 구성 요소가 자신의 안녕을 유지하...,0,[,',국,가,['국가가 안정성을 제공하지 못함으로써 사회의 각 구성 요소가 자신의 안녕을 유지하...,592
3257,다음 중 코로나바이러스의 복제 주기에서 특이한 특징은 무엇입니까?,1,"['모든 RNA가 공통된 3' 말단에서 끝나고 중첩된 세트 전사체를 생성합니다', ...",1,[,',모,든,['모든 RNA가 공통된 3' 말단에서 끝나고 중첩된 세트 전사체를 생성합니다'- ...,112
3417,안전한 물에 대한 접근이 제한되고 교육 수준이 낮은 국가의 신생아는:,0,"[6개월 동안 분유만 먹여야 한다, 6개월 동안 모유만 먹여야 한다, 첫 6개월 동...",1,6개월 동안 분유만 먹여야 한다,6개월 동안 모유만 먹여야 한다,첫 6개월 동안 모유와 다른 음식을 함께 먹여야 한다,0,"[6개월 동안 분유만 먹여야 한다, 6개월 동안 모유만 먹여야 한다, 첫 6개월 동...",3
4411,"Scott (1990)에 따르면, 문서가 '진본'으로 간주되기 위해서는:",0,"['정치적 편견 없이 진실되고 정확한 설명이어야 한다', '생존하지 못한 유사한 모...",3,[,',정,치,"['정치적 편견 없이 진실되고 정확한 설명이어야 한다', '생존하지 못한 유사한 모...",143
5439,'자산'이라는 용어는 다음을 의미합니다:,0,['가난한 사람들이 상대적으로 박탈감을 느끼게 하는 문화적으로 가치 있는 상품과 생...,2,[,',가,난,['가난한 사람들이 상대적으로 박탈감을 느끼게 하는 문화적으로 가치 있는 상품과 생...,139
7115,"에너지 안보와 석유 생산의 맥락에서, 다음 중 '안정성'을 정의하지 않는 것은 무엇...",1,"['석유 생산국의 체제에 대한 도전이 없는 상태.', '생산, 수출 및 공급의 현상...",2,[,',석,유,"['석유 생산국의 체제에 대한 도전이 없는 상태.', '생산- 수출 및 공급의 현상...",162
8856,'에너지 안보 넥서스'의 중요한 결과는 무엇입니까?,1,['남반구의 에너지 매장량을 통제하려는 시도는 핵심 강대국 간 협력의 지속 가능성에...,0,[,',남,반,['남반구의 에너지 매장량을 통제하려는 시도는 핵심 강대국 간 협력의 지속 가능성에...,271


In [77]:
df_p_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9000 entries, 0 to 8999
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   문제       9000 non-null   object
 1   cluster  9000 non-null   int64 
 2   선택지      9000 non-null   object
 3   답안       9000 non-null   int64 
 4   0        9000 non-null   object
 5   1        9000 non-null   object
 6   2        9000 non-null   object
 7   3        9000 non-null   object
 8   __선택지    9000 non-null   object
dtypes: int64(2), object(7)
memory usage: 632.9+ KB


In [84]:
df_p_data.to_csv('temp_train_preprocessing.csv', index=False, encoding='utf-8')

In [85]:
df_p_data.tail(20)

,문제,cluster,선택지,답안,0,1,2,3,__선택지,문항개수
8980,안보화 과정은 어떻게 완료됩니까?,1,[안보화 과정은 정치 영역에 포함되지 않은 문제가 정치화되어 정치 시스템 내에서 관...,3,안보화 과정은 정치 영역에 포함되지 않은 문제가 정치화되어 정치 시스템 내에서 관리...,안보화 행위자는 정치 영역 밖에 있는 문제를 안보화할 수 있다고 주장할 수 있습니다...,안보화는 문제를 정치화된 상태에서 정상적인 정치적 의사 결정 활동의 영역을 넘어선 ...,문제는 정치화된 영역에서 안보화된 영역으로 이동될 때 안보 문제로 구성될 수 있습니...,[안보화 과정은 정치 영역에 포함되지 않은 문제가 정치화되어 정치 시스템 내에서 관...,4
8981,어떤 지용성 비타민을 보충하면 심혈관 질환을 줄이는 것으로 나타났습니까?,1,"[비타민 A와 D, 비타민 A와 K, 비타민 D와 E, 위의 어느 것도 아님]",3,비타민 A와 D,비타민 A와 K,비타민 D와 E,위의 어느 것도 아님,"[비타민 A와 D, 비타민 A와 K, 비타민 D와 E, 위의 어느 것도 아님]",4
8982,"다음 정보와 관련된 질문입니다.\n출처 1:\n""더욱이, 경이시여, 우리 왕국에는 ...",1,"[서아프리카 해안에 대한 포르투갈 탐험, 사하라 이남 아프리카에서의 가톨릭 선교 활...",3,서아프리카 해안에 대한 포르투갈 탐험,사하라 이남 아프리카에서의 가톨릭 선교 활동,사하라 이남 아프리카에서의 국가 형성,사하라 이남 아프리카에서의 노예 무역 발전,"[서아프리카 해안에 대한 포르투갈 탐험, 사하라 이남 아프리카에서의 가톨릭 선교 활...",4
8983,직장 검사에 대해 다음 중 참인 것은 무엇입니까?,1,"[환자가 보호자를 거부하면 의사가 이를 무시할 수 있다, 직장 검사는 신경학적 검사...",3,환자가 보호자를 거부하면 의사가 이를 무시할 수 있다,직장 검사는 신경학적 검사에서 아무런 가치가 없다,환자의 최적 위치는 오른쪽으로 누워 무릎을 펴는 것이다,직장 검사는 일부 위장관 질환 평가에 적합하다,"[환자가 보호자를 거부하면 의사가 이를 무시할 수 있다, 직장 검사는 신경학적 검사...",4
8984,다음 중 가수분해의 예가 아닌 것은 무엇입니까?,1,"[지방을 지방산과 글리세롤로 전환, 단백질을 아미노산으로 전환, 전분을 단당으로 전...",3,지방을 지방산과 글리세롤로 전환,단백질을 아미노산으로 전환,전분을 단당으로 전환,피루브산을 아세틸 CoA로 전환,"[지방을 지방산과 글리세롤로 전환, 단백질을 아미노산으로 전환, 전분을 단당으로 전...",4
8985,"2020년 미국의 일반적인 도덕 기준에 따르면, 두 가지 시나리오 중 어느 시나리오...",2,"[잘못됨, 잘못됨, 잘못됨, 잘못되지 않음, 잘못되지 않음, 잘못됨, 잘못되지 않음...",2,"잘못됨, 잘못됨","잘못됨, 잘못되지 않음","잘못되지 않음, 잘못됨","잘못되지 않음, 잘못되지 않음","[잘못됨, 잘못됨, 잘못됨, 잘못되지 않음, 잘못되지 않음, 잘못됨, 잘못되지 않음...",4
8986,환 Z x Z의 표수를 구하십시오.,0,"[0, 3, 12, 30]",0,0,3,12,30,"[0, 3, 12, 30]",4
8987,1차 데이터는,0,"[마케팅 연구를 위해 처음으로 수집된 데이터입니다., 2차 데이터보다 획득하는 데 ...",0,마케팅 연구를 위해 처음으로 수집된 데이터입니다.,2차 데이터보다 획득하는 데 시간이 덜 걸리고 비용이 적게 듭니다.,정부 조사 결과가 새로 발표된 것입니다.,적절한 자원 배분을 보장하기 위한 전략적 계획 활동의 일부입니다.,"[마케팅 연구를 위해 처음으로 수집된 데이터입니다., 2차 데이터보다 획득하는 데 ...",4
8988,정신과 의사가 7년 전에 평가된 환자에 대한 보고서 사본을 요청하는 서명된 환자 정...,0,[시간이 너무 많이 경과했기 때문에 보고서를 보내기를 거부하는 편지를 정신과 의사에...,3,시간이 너무 많이 경과했기 때문에 보고서를 보내기를 거부하는 편지를 정신과 의사에게...,요청한 대로 보고서를 보내야 한다,보고서를 보내기 전에 환자에게 직접 연락해야 한다,보고서가 이제는 쓸모없을 수 있음을 나타내는 표지를 첨부하여 보고서를 보내야 한다,[시간이 너무 많이 경과했기 때문에 보고서를 보내기를 거부하는 편지를 정신과 의사에...,4
8989,"실서증, 계산불능증, 손가락 인지불능증, 좌우 혼동은 다음 중 어느 질환의 증상입니까?",1,"[크로이츠펠트-야콥병, 애디슨병, 게르스트만 증후군, 게슈빈드 증후군]",2,크로이츠펠트-야콥병,애디슨병,게르스트만 증후군,게슈빈드 증후군,"[크로이츠펠트-야콥병, 애디슨병, 게르스트만 증후군, 게슈빈드 증후군]",4
